# Задание 2. SQL

In [1]:
import pandahouse as ph

In [2]:
# Словарь с нужными параметрами для подключения к Clickhouse
# для подключения к default:
connection_default = {'host': 'http://clickhouse.beslan.pro:8080',
'database':'default',
'user':'student',
'password':'dpo_python_2020'
}

2.1 Очень усердные ученики.

2.1.1 Условие

Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. Каждое такое маленькое задание называется "горошиной".

Назовём очень усердным учеником того пользователя, который хотя бы раз за текущий месяц правильно решил 20 горошин.

2.1.2 Задача

Дана таблица default.peas.

Необходимо написать оптимальный запрос, который даст информацию о количестве очень усердных студентов за март 2020 года.

NB! Под усердным студентом мы понимаем студента, который правильно решил 20 задач за текущий месяц.

In [3]:
# Пишем запрос
# В таблице нет данных за март 2020 года, поэтому запрос возвращает 0!
q = '''
SELECT COUNT(a.st_id) as count_diligent_st
FROM
(
    SELECT st_id
    FROM default.peas
    WHERE
        timest >= '2020-03-01' AND
        timest <= '2020-03-31'
    GROUP BY st_id
    HAVING SUM(correct) >= 20
) a
'''
# отправляем запрос и записываем результат в пандасовский датафрейм
q_first = ph.read_clickhouse(query=q, connection=connection_default)
q_first

,count_diligent_st
0,0


2.2 Оптимизация воронки

2.2.1 Условие

Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.

2.2.2 Задача

Дана таблицы: default.peas, default.studs и default.final_project_check:

Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:

ARPU, 
ARPAU,
CR в покупку, 
СR активного пользователя в покупку,
CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике

ARPU считается относительно всех пользователей, попавших в группы.

Активным считается пользователь, за все время решивший больше 10 задач правильно в любых дисциплинах.

Активным по математике считается пользователь, за все время решивший 2 или больше задач правильно по математике.

In [4]:
# Пишем запрос
# Запрос состоит из 5 главных подзапросов, в которых я отдельно нахожу каждую метрику. В конце склеиваю таблицы.
q = '''
SELECT 
    ARPU_t.test_grp as test_grp,
    ARPU_t.ARPU as ARPU,
    ARPAU_t.ARPAU as ARPAU,
    CR_t.CR as CR,
    CR_active_t.CR_active as CR_active,
    CR_active_math_t.CR_active_math as CR_active_math
FROM 
    (
        SELECT 
            test_grp,
            SUM(money)/COUNT(DISTINCT st_id) as ARPU 
        FROM default.studs as ds
        LEFT JOIN  
            default.final_project_check as df
        USING(st_id)
        GROUP BY test_grp
    ) ARPU_t
JOIN  
    (
        SELECT 
            test_grp,
            SUM(money)/COUNT(DISTINCT st_id) as ARPAU 
        FROM default.studs as ds
        LEFT JOIN  
            default.final_project_check as df
        USING(st_id)
        WHERE
            ds.st_id IN (
                SELECT st_id
                FROM default.peas
                GROUP BY st_id
                HAVING SUM(correct) > 10
            ) as active_st
        GROUP BY test_grp
    ) ARPAU_t
ON ARPU_t.test_grp = ARPAU_t.test_grp
JOIN  
    (
        SELECT 
        test_grp,
        AVG(isBuy) as CR
        FROM
            (
                SELECT 
                    ds.st_id, test_grp,
                    CASE
                    WHEN SUM(money) > 0 THEN 1
                    ELSE 0
                END AS isBuy
                FROM default.studs as ds
                LEFT JOIN  
                    default.final_project_check as df
                USING(st_id)
                GROUP BY ds.st_id, test_grp
            ) a  
        GROUP BY test_grp
    ) CR_t
ON ARPU_t.test_grp = CR_t.test_grp
JOIN  
    (
        SELECT 
            test_grp,
            AVG(isBuy) as CR_active
        FROM
            (
                SELECT 
                    ds.st_id, test_grp,
                    CASE
                    WHEN SUM(money) > 0 THEN 1
                    ELSE 0
                END AS isBuy
                FROM default.studs as ds
                LEFT JOIN  
                    default.final_project_check as df
                USING(st_id)
                WHERE 
                ds.st_id IN 
                (
                    SELECT st_id
                    FROM default.peas
                    GROUP BY st_id
                    HAVING SUM(correct) > 10
                ) as active_st
                GROUP BY ds.st_id, test_grp
            ) a  
        GROUP BY test_grp
    ) CR_active_t 
ON ARPU_t.test_grp = CR_active_t.test_grp
JOIN  
    (
        SELECT 
            test_grp,
            AVG(isBuy) as CR_active_math
        FROM
            (
                SELECT 
                    a.st_id as st_id,
                    ds.test_grp as test_grp,
                    CASE
                        WHEN SUM(money) > 0 THEN 1
                        ELSE 0
                    END AS isBuy
                FROM 
                    (
                        SELECT st_id
                        FROM default.peas
                        WHERE subject = 'Math'
                        GROUP BY st_id
                        HAVING SUM(correct) > 1
                    ) a
                LEFT JOIN  
                    (
                        SELECT *
                        FROM default.final_project_check
                        WHERE subject = 'Math'
                    ) b
                USING(st_id)
                LEFT JOIN 
                    default.studs as ds 
                ON 
                    a.st_id = ds.st_id
                GROUP BY a.st_id, ds.test_grp
            ) ab
        GROUP BY test_grp
    ) CR_active_math_t 
ON ARPU_t.test_grp = CR_active_math_t.test_grp
'''
# отправляем запрос и записываем результат в пандасовский датафрейм
q_second = ph.read_clickhouse(query=q, connection=connection_default)
q_second

,test_grp,ARPU,ARPAU,CR,CR_active,CR_active_math
0,control,4540.983607,10393.700787,0.049180,0.110236,0.061224
1,pilot,11508.474576,29739.583333,0.108475,0.260417,0.095238
